In [1]:
# Imports
import refinitiv.dataplatform.eikon as ek
import pandas as pd
import time

In [2]:
# Utils
def get_ric(symbol):
    """
    symbol: the symbol of the company name in your portfolio.  
    Find the best possible RIC match for company names.
    returns: ric if there is a match, or initial company name. 
    """
    possible_rics = ek.json_requests.send_json_request('SymbologySearch',{'symbols': [symbol], 'from': 'ticker', 'to': ['RIC'], 'bestMatchOnly': False})
    try:
        return possible_rics.json().get("mappedSymbols")[0].get("RICs")[0]
    except:
        return symbol
    
def get_isin_from_ric(ric):
    """ 
    Find the ISIN match for your input RIC.
    """
    isin = ek.get_symbology(ric, from_symbol_type="RIC", to_symbol_type="ISIN")
    try:
        return isin.ISIN[0]
    except:
        return "N/A"
    

def get_isin_from_sedol(sedol):
    """ 
    Find the ISIN match for your input sedol.
    """
    isin = ek.get_symbology(sedol, from_symbol_type="SEDOL", to_symbol_type="ISIN")
    try:
        return isin.ISIN[0]
    except:
        return "N/A"
    
def format_df(esg_df, esg_df_bis, year):
    test = esg_df_bis[esg_df_bis.year == year].copy().drop(columns=["year"])
    test.columns = ["Instrument", "Environmental Pillar Score Grade "+str(year), "Environmental Pillar Score "+str(year)]
    return pd.merge(esg_df, test, how="left", left_on="Instrument", right_on="Instrument")

Set the API key :

In [3]:
app_key = "213f70312b3740cfa9f65def2171d679808d0131"#"467e1d8d1e624dfc834b60e91bba50e474bea063"
ek.set_app_key(app_key)

#### First, we get the Reuters Index Codes from the portfolio company symbols.

In [7]:
input_path = "data/" #"../inputs/"
output_path = "output/"
filename = "allocation210420.csv"
#portfolio = pd.read_excel(input_path+filename)#.drop(columns=["Unnamed: 0"])#header=None)
portfolio = pd.read_csv(input_path+filename, sep=";")

In [10]:
portfolio = portfolio[portfolio.Sleeve=="equity"].copy()

Now, get the rics from the symbols using the eikon api :

In [13]:
#rics = [get_ric(symbol) for symbol in symbols]
#isins = [get_isin_from_ric(ric) for ric in rics]
isins = list(portfolio.ISIN)

In [14]:
#sedols = portfolio.loc[:,"SEDOL"]

In [16]:
#isins = [get_isin_from_sedol(sedol) for sedol in sedols]

#### Next, we retrieve the corresponding ESG data. 

Find the field names you are interested in by looking at the Data Item Browser (type it in the search bar).

In [17]:
help(ek.get_data)

Help on function get_data in module refinitiv.dataplatform.legacy.data_grid:

get_data(instruments, fields, parameters=None, field_name=False, raw_output=False, debug=False, raw_response=False)
    Returns a pandas.DataFrame with fields in columns and instruments as row index
    
    Parameters
    ----------
    instruments: string or list
        Single instrument or list of instruments to request.
    
    fields: string, dictionary or list of strings and/or dictionaries.
        List of fields to request.
    
        Examples:
    
        - 'TR.PriceClose'
        - {'TR.GrossProfit': { 'params':{ 'Scale': 6, 'Curn': 'EUR' }}
        - {'TR.GrossProfit': { 'params':{ 'Scale': 6, 'Curn': 'EUR' },sort_dir:'desc'}
        - ['TR.PriceClose','TR.PriceOpen']
        - [{'TR.PriceClose':  {'sort_dir':asc,sort_priority:1}},{'TR.PriceOpen':  {'sort_dir':asc,sort_priority:0}}
        
        You can use the legacy TR_Field to build the fields:
    
        >>> fields = [ek.TR_Field('tr.

In [22]:
fields = [
    'TR.TRESGScoreGrade',
    'TR.EnvironmentPillarScoreGrade',
    'TR.SocialPillarScoreGrade',
    'TR.GovernancePillarScoreGrade',
    'TR.AnalyticCO2ScoreGrade',
    'TR.TRESGInnovationScoreGrade',
    'TR.TRESGCSRStrategyScoreGrade',
    #'TR.TRESGCControversiesScore',
    #'TR.TRESGScoreGrade',
    #'TR.EnvironmentPillarScoreGrade',
    #'TR.SocialPillarScoreGrade',
    #'TR.GovernancePillarScoreGrade',
    #'TR.TRESGCControversiesScoreGrade',
   # 'TR.PolicyEmissions',
   # 'TR.TargetEmissions', # SBTi
   # 'TR.EmissionReductionTargetPctage', # SBTi
   # 'TR.CO2EstimationMethod', 
   # 'TR.EmissionReductionTargetYear', # SBTi
   # 'TR.PolicyEmissionsScore',
   # 'TR.TargetEmissionsScore',
   # 'TR.TRESGResourceUseScore',
   # 'TR.CO2EmissionTotal',
   # 'TR.CO2DirectScope1',
   # 'TR.CO2IndirectScope2',
   # 'TR.CO2IndirectScope3',
   # 'TR.AnalyticEstimatedCO2Total',
   # 'TR.AnalyticCO2',   
   # 'TR.EnergyUseTotal',
   # 'TR.AnalyticEnergyUse',
   # 'TR.TRESGInnovationScoreGrade',
    #'TR.TRESGCSRStrategyScoreGrade',
   # 'TR.EnergyUseScore',
]

environmental_fields = [
    'TR.EnvironmentPillarScoreGrade(SDate=0,EDate=-3,Period=FY0,Frq=FY).date',
    'TR.EnvironmentPillarScoreGrade(SDate=0,EDate=-3,Period=FY0,Frq=FY)',
    'TR.EnvironmentPillarScore(SDate=0,EDate=-3,Period=FY0,Frq=FY).date',
    'TR.EnvironmentPillarScore(SDate=0,EDate=-3,Period=FY0,Frq=FY)',
]

In [23]:
esg_df, err = ek.get_data(instruments = isins, fields=fields)

In [24]:
esg_df_bis, err = ek.get_data(instruments = isins, fields=environmental_fields)

#### Then, add the ESG data to the initial portfolio. 

In [25]:
esg_df

,Instrument,ESG Score Grade,Environmental Pillar Score Grade,Social Pillar Score Grade,Governance Pillar Score Grade,Environmental Innovation Score Grade,CSR Strategy Score Grade
0,US4581401001,A,A,A,A+,A-,A+
1,US3886891015,B,B+,C+,B+,C,A+
2,US3453708600,A,A+,A,B,A,A+
3,US2435371073,B-,B-,B,C+,D+,B+
4,US0382221051,A-,A-,B+,A-,A-,B+
5,US0394831020,A-,A-,A-,A,A,A+
6,US0367521038,A-,A-,A,A-,D-,B-
7,US0200021014,A-,A-,A-,B+,A-,B+
8,US00508Y1029,B-,B,B-,C,B+,B
9,US1170431092,B+,B-,B+,A,C+,A-


In [26]:
esg_df.columns

Index(['Instrument', 'ESG Score Grade', 'Environmental Pillar Score Grade',
       'Social Pillar Score Grade', 'Governance Pillar Score Grade',
       'Environmental Innovation Score Grade', 'CSR Strategy Score Grade'],
      dtype='object')

In [27]:
#esg_df["Symbol"] = symbols
#esg_df["SEDOL"] = sedols
esg_df["company_isin"] = isins

In [28]:
portfolio_df = portfolio.copy()#iloc[:-12,:]

In [29]:
esg_df_bis["year"] = pd.to_datetime(esg_df_bis.Date.iloc[:,0]).dt.year

In [30]:
esg_df_bis = esg_df_bis.drop(columns=["Date"])

In [31]:
years = list(esg_df_bis.year.value_counts().index)
for year in years:
    esg_df = format_df(esg_df, esg_df_bis, year)

In [39]:
esg_df = esg_df.rename(columns={"Instrument":"ISIN"})

In [40]:
out = pd.merge(portfolio_df, esg_df, how="left", left_on="ISIN", right_on=["ISIN"])

In [41]:
out

,Date,ISIN,instrument_name,Quantity,Valuation,Weight,Asset Class,Sector,Subsector,Country,...,Environmental Pillar Score Grade 2017,Environmental Pillar Score 2017,Environmental Pillar Score Grade 2019,Environmental Pillar Score 2019,Environmental Pillar Score Grade 2016,Environmental Pillar Score 2016,Environmental Pillar Score Grade 2020,Environmental Pillar Score 2020,Environmental Pillar Score Grade 2015,Environmental Pillar Score 2015
0,20/04/2021,US4581401001,INTEL CORP COM USD0.001,32601,"2110914,8","0,019652104",Actions,Information Technology,Semiconductors,USA,...,A,91.441327,A,90.112512,A,90.726723,<NA>,NaN,<NA>,NaN
1,20/04/2021,US3886891015,GRAPHIC PACKAGING HOLDING CO,60000,1110600,"0,010339416",Actions,Materials,Paper Packaging,USA,...,B-,56.839967,B+,71.735113,C-,27.555011,<NA>,NaN,<NA>,NaN
2,20/04/2021,US3453708600,FORD MOTOR,58492,"709507,94","0,006605347",Actions,Consumer Discretionary,Automobile Manufacturers,USA,...,A,85.528925,A+,92.032877,A,86.101393,<NA>,NaN,<NA>,NaN
3,20/04/2021,US2435371073,DECKERS OUTDOOR CORP COM,5200,1773512,"0,016510965",Actions,Consumer Discretionary,Footwear,USA,...,C+,42.787524,C+,47.280020,<NA>,NaN,B-,55.789356,<NA>,NaN
4,20/04/2021,US0382221051,APPLIED MATERIALS NPV,14200,1898966,"0,017678913",Actions,Information Technology,Semiconductor Equipment,USA,...,B+,71.361961,A-,79.300546,B+,68.962486,<NA>,NaN,<NA>,NaN
5,20/04/2021,US0394831020,ARCHER DANIELS MIDLAND NPV,55327,"3289190,2","0,03062156",Actions,Consumer Staples,Agricultural Products,USA,...,A-,76.834243,A-,79.523350,B-,55.015445,<NA>,NaN,<NA>,NaN
6,20/04/2021,US0367521038,ANTHEM INC UN USD,7198,"2703712,8","0,02517091",Actions,Health Care,Managed Health Care,USA,...,B+,74.219858,A-,80.521592,B+,67.169059,<NA>,NaN,<NA>,NaN
7,20/04/2021,US0200021014,ALLSTATE CORP,3286,"401549,2","0,003738325",Actions,Financials,Property & Casualty Insurance,USA,...,A-,82.377392,A-,79.098106,A-,83.022599,<NA>,NaN,<NA>,NaN
8,20/04/2021,US00508Y1029,ACUITY BRANDS INC,8450,"1447400,5","0,013474947",Actions,Industrials,Electrical Components & Equipment,USA,...,C,37.228769,B,59.847476,<NA>,NaN,B,61.373732,<NA>,NaN
9,20/04/2021,US1170431092,BRUNSWICK,19000,1996330,"0,018585347",Actions,Consumer Discretionary,Leisure Products,USA,...,D+,22.727273,B-,50.879765,D+,22.727273,<NA>,NaN,<NA>,NaN


In [42]:
out.columns

Index(['Date', 'ISIN', 'instrument_name', 'Quantity', 'Valuation', 'Weight',
       'Asset Class', 'Sector', 'Subsector', 'Country', 'Sleeve',
       'ESG Score Grade', 'Environmental Pillar Score Grade',
       'Social Pillar Score Grade', 'Governance Pillar Score Grade',
       'Environmental Innovation Score Grade', 'CSR Strategy Score Grade',
       'company_isin', 'Environmental Pillar Score Grade 2018',
       'Environmental Pillar Score 2018',
       'Environmental Pillar Score Grade 2017',
       'Environmental Pillar Score 2017',
       'Environmental Pillar Score Grade 2019',
       'Environmental Pillar Score 2019',
       'Environmental Pillar Score Grade 2016',
       'Environmental Pillar Score 2016',
       'Environmental Pillar Score Grade 2020',
       'Environmental Pillar Score 2020',
       'Environmental Pillar Score Grade 2015',
       'Environmental Pillar Score 2015'],
      dtype='object')

In [44]:
out.to_csv(output_path+"portfolio_210421.csv")